In [ ]:
# Libraries
import pandas as pd   # to load and transform data
import numpy as np    # for math/stat calculations
import altair as alt
import json 

In [ ]:
# Read in data
flights = pd.read_json("https://github.com/byuidatascience/data4missing/raw/master/data-raw/flights_missing/flights_missing.json")


In [ ]:
# Clean Data
# flights= (
    # flights
    # .replace(['n/a', "1500+", -999], np.nan)
    # .fillna(method = 'ffill')
# )
# flights_dat.head()



## Grand Question 1 
Which airport has the worst delays? 
- Discuss how you chose to define “worst”
- Your answer should also __include a table__ that lists (for each airport): 
    - the total number of flights 
    - total number of delayed flights 
    - proportion of delayed flights 
    - average delay time in hours

To define the worst delays, we will find the airport that has the highest percentage of delayed flights

In [ ]:
months = ["January", "Febuary", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

gq1 = (
flights.groupby('airport_code')
       .agg(total_flights = ('num_of_flights_total', sum), 
            total_delays = ('num_of_delays_total', sum), 
            total_delay_minutes = ('minutes_delayed_total', sum))
       .assign(prop_delays = lambda X: X.total_delays / X.total_flights, 
               avg_delay_hours = lambda X: X.total_delay_minutes / X.total_delays / 60, 
               percent_delays = lambda X: X.prop_delays * 100)
       .sort_values('prop_delays', ascending = False)
       .reset_index()
)
# gq1.pd.dataframe()
print(gq1.to_markdown())

chart1 = (
alt.Chart(gq1)
        .mark_bar().encode(
                x = alt.X('prop_delays:Q', title = "Percent of Flights Delayed", axis=alt.Axis(format = '%')), 
                y = alt.Y('airport_code:O', title = "Airport Code", sort='-x')
        )
        .properties(
                title = "San Fran Exects Delays"
        )
)
chart1

## Grand Question 2 
What is the best month to fly if you want to avoid delays of any length?

In [ ]:
flights3 = flights.replace(['n/a', "1500+", -999], np.nan)

gq2 = (
flights
     .replace(to_replace = "Febuary", value = "February")
     .dropna(subset = ['month'])
     .query("month != 'n/a'")
     .groupby('month')
     .agg(total_flights = ('num_of_flights_total', sum), 
          total_delays = ('num_of_delays_total', sum), 
          total_delay_minutes = ('minutes_delayed_total', sum)
     )
     .assign(total_delay_hours = lambda x: x.total_delay_minutes / 60)
     .reset_index()            
)

gq2 = (gq2.assign(month_number = pd.to_datetime(gq2.month, format = "%B").dt.month, 
                  month_date = pd.to_datetime(gq2.month, format = "%B"))
          .sort_values('month_number', ascending = False)
          .reset_index())

(alt.Chart(gq2)
.mark_bar()
.encode(
     x = alt.X('total_delay_hours:Q', title = 'Delays in Hours'), 
     y = alt.Y('month(month_date):O', title = 'Month of Year' 
     # sort = 'month_number'
     # sort = alt.EncodingSortField(field="months", order='ascending')
     ))
.properties(title = "November & September: Least Delays")
)

## Grand Question 3
According to the BTS website, the “Weather” category only accounts for severe weather delays. Mild weather delays are not counted in the “Weather” category, but are actually included in both the “NAS” and “Late-Arriving Aircraft” categories. 

**Your job is to create a new column that calculates the total number of flights delayed by weather (both severe and mild).** 

You will need to replace all the missing values in the Late Aircraft variable with the mean. 

Use these three rules for your calculations:

- 100% of delayed flights in the Weather category are due to weather.
- 30% of delayed flights in the Late-Arriving category are due to weather.
- From April to August, 40% of delayed flights in the NAS category are due to weather. The rest of the months, the proportion rises to 65%.

In [11]:
late_aircraft_avg = flights.num_of_delays_late_aircraft.replace(-999, np.nan).mean()
q3 = flights
q3.num_of_delays_late_aircraft.replace(-999, late_aircraft_avg, inplace = True)
# q3

q3_new = (
    q3
    .assign(total_number_weather_delayed_flights = lambda x: x.         num_of_delays_late_aircraft * 0.3 + 
         x.num_of_delays_nas * 0.4 +
         x.num_of_delays_weather
    )
    # .sort_values('total_number_weather_delayed_flights')
    # .filter('total_number_weather_delayed_flights')
)
print(q3_new.filter(["airport_code", "month", "year", "total_number_weather_delayed_flights", ]).head().to_markdown())
# q3.head()



|    | airport_code   | month   |   year |   total_number_weather_delayed_flights |
|---:|:---------------|:--------|-------:|---------------------------------------:|
|  0 | ATL            | January |   2005 |                                2619.93 |
|  1 | DEN            | January |   2005 |                                 885.4  |
|  2 | IAD            | January |   2005 |                                 736.4  |
|  3 | ORD            | January |   2005 |                                3148.5  |
|  4 | SAN            | January |   2005 |                                 515.2  |


## Grand Question 4
Create a barplot showing the proportion of all flights that are delayed by weather at each airport. Discuss what you learn from this graph.

In [12]:
(
    alt.Chart(q3_new)
    .mark_bar()
    .encode(x = alt.X('total_number_weather_delayed_flights:Q', title = "Amount of Weather Delays"), 
            y = alt.Y('airport_code:O', title = "Airport Code"))
)

alt.Chart(...)

In [13]:
weather = q3.assign(
    severe = q3.num_of_delays_weather, 
    mild_late = .3 * q3.num_of_delays_late_aircraft, 
    mild_nas = np.where(q3.month.isin(["April", "May", "June", "July", "August"]), 
                        .4 * q3.num_of_delays_nas, 
                        .65 * q3.num_of_delays_nas), 
    total_weather = lambda x: x.severe + x.mild_late + x.mild_nas, 
)

In [14]:
q4_data = (weather.groupby("airport_code")
                .agg(total_weather_delays = ('total_weather', sum),
                     total_flights = ("num_of_flights_total", sum))
                .assign(per_weather_delays = lambda x: x.total_weather_delays / x.total_flights)
                .reset_index())
q4_data.head()

,airport_code,total_weather_delays,total_flights,per_weather_delays
0,ATL,314800.623982,4430047,0.071060
1,DEN,149106.950000,2513974,0.059311
2,IAD,50842.650000,851571,0.059705
3,ORD,309954.074887,3597588,0.086156
4,SAN,48920.550000,917862,0.053298


## Grand Question 5 
Fix all of the varied missing data types in the data to be consistent (all missing values should be displayed as “NaN”) and convert the new data back to JSON format. Include one record example from your exported JSON file that has a missing value.

In [15]:
# Clean Data
flights= (
    flights
    .replace(['n/a', "1500+", -999], np.nan)
)

json_data = flights.to_json()
json_data = flights.to_json(orient = "records")
json_object = json.loads(json_data)
json_formatted_str = json.dumps(json_object, indent = 4)
print(json_formatted_str)

[
    {
        "airport_code": "ATL",
        "airport_name": "Atlanta, GA: Hartsfield-Jackson Atlanta International",
        "month": "January",
        "year": 2005.0,
        "num_of_flights_total": 35048,
        "num_of_delays_carrier": null,
        "num_of_delays_late_aircraft": 1109.1040723982,
        "num_of_delays_nas": 4598,
        "num_of_delays_security": 10,
        "num_of_delays_weather": 448,
        "num_of_delays_total": 8355,
        "minutes_delayed_carrier": 116423.0,
        "minutes_delayed_late_aircraft": 104415,
        "minutes_delayed_nas": 207467.0,
        "minutes_delayed_security": 297,
        "minutes_delayed_weather": 36931,
        "minutes_delayed_total": 465533
    },
    {
        "airport_code": "DEN",
        "airport_name": "Denver, CO: Denver International",
        "month": "January",
        "year": 2005.0,
        "num_of_flights_total": 12687,
        "num_of_delays_carrier": "1041",
        "num_of_delays_late_aircraft": 928.0,
      